# PAL case study

### Notebook helper function for pretty output

In [ ]:
from IPython.display import Markdown
from IPython.display import display


def print_md(markdown_printable):
    """Pretty render Markdown."""
    display(Markdown(str(markdown_printable)))


def print_yaml_file(file_path):
    """Pretty render yaml file."""
    with open(file_path) as f:
        print_md(f"```yaml\n{f.read()}\n```")

### Plotting functions ([`pyglotaran_extras`](https://github.com/s-weigand/pyglotaran-extras/commit/20da3593105fb839f86e668dc12dc9ca87c3b9ce) + `matplotlib`)

In [ ]:
import matplotlib.pyplot as plt
from pyglotaran_extras.plotting.plot_overview import plot_overview
from pyglotaran_extras.plotting.style import PlotStyle

plot_style = PlotStyle()
plt.rc("axes", prop_cycle=plot_style.cycler)
plt.rcParams["figure.figsize"] = (21, 14)

### Analysis functions

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.io import load_dataset
from glotaran.io import load_model
from glotaran.io import load_parameters
from glotaran.project.scheme import Scheme

## Global Analysis

### Read data

In [ ]:
dataset = load_dataset(
    "PAL_DCMU_80uM_WL_SF_700_ma_tr2_gain50_100uW_590nmfilter_21C_400nm_AV_bc_sh_sel_620_830.ascii"
)
dataset.data.plot(x="time")

### Load model and parameters

In [ ]:
PAL_closed_global_model = load_model("models/PAL-closed-global-model.yml")
PAL_closed_global_parameters = load_parameters("models/PAL-closed-global-parameters.yml")
print_md(PAL_closed_global_model.validate(parameters=PAL_closed_global_parameters))

In [ ]:
print_md(PAL_closed_global_model)

### Create scheme and optimize it

In [ ]:
PAL_closed_global_scheme = Scheme(
    PAL_closed_global_model,
    PAL_closed_global_parameters,
    {"dataset1": dataset},
    non_negative_least_squares=True,
)
PAL_closed_global_result = optimize(PAL_closed_global_scheme)

In [ ]:
PAL_closed_global_result.data["dataset1"]

### Result plots

In [ ]:
plot_overview(PAL_closed_global_result.data["dataset1"], linlog=False);

In [ ]:
print_md(PAL_closed_global_result.optimized_parameters)

## Target Analysis

### Load model and parameters

In [ ]:
PAL_closed_target_model = load_model("models/PAL-closed-target-model.yml")
PAL_closed_target_parameters = load_parameters("models/PAL-closed-target-parameters.yml")
print_md(PAL_closed_target_model.validate(parameters=PAL_closed_target_parameters))

In [ ]:
compartments = PAL_closed_target_model.initial_concentration["input1"].compartments
print_md(
    PAL_closed_target_model.megacomplex["mc1"]
    .full_k_matrix(PAL_closed_target_model)
    .matrix_as_markdown(compartments)
    .replace("0.0000e+00", "")
)

In [ ]:
print_md(PAL_closed_target_model)

### Create scheme and optimize it

In [ ]:
PAL_closed_target_scheme = Scheme(
    PAL_closed_target_model,
    PAL_closed_target_parameters,
    {"dataset1": dataset},
    non_negative_least_squares=True,
)
PAL_closed_target_result = optimize(PAL_closed_target_scheme)

In [ ]:
PAL_closed_target_result.data["dataset1"]

### Result plots

In [ ]:
plot_overview(PAL_closed_target_result.data["dataset1"], linlog=False);

In [ ]:
print_md(PAL_closed_target_result.optimized_parameters)